In [1]:
import random, numpy as np, nltk, json, pickle, tensorflow as tf

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

# Télécharger 'punkt' pour s'assurer que la tokenisation fonctionne correctement
# nltk.download('punkt_tab') 

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Charger les fichiers
intents = json.loads(open('../intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Charger le modèle
model = load_model('chatbot_model.h5')

### Fonction pour traiter l'entrée utilisateur
def clean_up_sentence(sentence):
    # Tokeniser et lemmatiser la phrase
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    # Créer un bag of words à partir de la phrase
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

### Prédiction d'intention
def predict_class(sentence, model):
    # Créer le bag of words à partir de la phrase d'entrée
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    # Trier les résultats par probabilité
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

### Générer une réponse
def get_response(intents_list, intents_json):
    # Obtenir l'intention la plus probable
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])

# Charger le fichier intents.json
with open('../intents.json') as file:
    intents_data = json.load(file)

# Simuler un message utilisateur
message = "Hello"

# Prédire l'intention
intents = predict_class(message, model)

# Générer une réponse (passer intents et le fichier intents.json)
response = get_response(intents, intents_data)

# Afficher la réponse
print("Chatbot: " + response)

### Boucle interactive continue
print("Le chatbot est prêt à répondre à vos questions ! Tapez 'quit' pour arrêter.")

while True:
    # Demander une entrée utilisateur
    message = input("Vous: ")
    
    # Arrêter si l'utilisateur tape 'quit'
    if message.lower() == "quit":
        print("Chatbot: Bye! Come back again soon.")
        break
    
    # Prédire l'intention
    intents = predict_class(message, model)
    
    # Générer une réponse (passer intents et le fichier intents.json)
    response = get_response(intents, intents_data)
    
    # Afficher la réponse
    print("Chatbot: " + response)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
Chatbot: Good to see you again
Le chatbot est prêt à répondre à vos questions ! Tapez 'quit' pour arrêter.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Chatbot: Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Chatbot: Hi! What can I do for you today?
Chatbot: Bye! Come back again soon.
